# Developing Word Embeddings

Rather than use pre-trained embeddings (as we did in the baseline_deep_dive notebook), we can train word embeddings using our own dataset. In this notebook, we demonstrate the training process for producing word embeddings using the word2vec, GloVe, and fastText models. We'll utilize the STS Benchmark dataset for this task. 

# Table of Contents
* [Data Loading and Preprocessing](#Load-and-Preprocess-Data)
* [Word2Vec](#Word2Vec)
* [fastText](#fastText)
* [GloVe](#GloVe)
* [Concluding Remarks](#Concluding-Remarks)

In [1]:
import gensim
import sys
import os

## Load and Preprocess Data

In [2]:
sys.path.append("../../../")  ## set the environment path
BASE_DATA_PATH = "../../../data"
SAVE_FILES_PATH = BASE_DATA_PATH + "/trained_word_embeddings/"

if not os.path.exists(SAVE_FILES_PATH):
    os.makedirs(SAVE_FILES_PATH)
    
from utils_nlp.dataset.preprocess import (
    to_lowercase,
    to_spacy_tokens,
    rm_spacy_stopwords,
)
from utils_nlp.dataset import stsbenchmark

In [3]:
# Produce a pandas dataframe for the training set
stsTrain = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="train")

In [4]:
# train preprocessing
df_low = to_lowercase(stsTrain)  # convert all text to lowercase
sts_tokenize = to_spacy_tokens(df_low)  # tokenize normally
sts_train = rm_spacy_stopwords(sts_tokenize)  # tokenize with removal of stopwords

In [5]:
#append together the sentence1_tokens and sentence2_tokens columns to get a list of all tokenized sentences
sentences = sts_train["sentence1_tokens"].append(sts_train["sentence2_tokens"])

In [6]:
sentences[:10]

0                       [a, plane, is, taking, off, .]
1            [a, man, is, playing, a, large, flute, .]
2    [a, man, is, spreading, shreded, cheese, on, a...
3                 [three, men, are, playing, chess, .]
4                 [a, man, is, playing, the, cello, .]
5                        [some, men, are, fighting, .]
6                             [a, man, is, smoking, .]
7               [the, man, is, playing, the, piano, .]
8    [a, man, is, playing, on, a, guitar, and, sing...
9    [a, person, is, throwing, a, cat, on, to, the,...
dtype: object

## Word2Vec

Word2vec is a predictive model for learning word embeddings from text. Word embeddings are learned such that words that share common contexts in the corpus will be close together in the vector space. There are two different model architectures that can be used to produce word2vec embeddings: continuous bag-of-words (CBOW) or continuous skip-gram. The former uses a window of surrounding words (the "context") to predict the current word and the latter uses the current word to predict the surrounding context words. See this [tutorial](https://www.guru99.com/word-embedding-word2vec.html#3) on word2vec for more detailed background on the model.

The gensim Word2Vec model has many different parameters (see [here](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) but the ones that are useful to know about are:  
- size: length of the word embedding/vector (defaults to 100)
- window: maximum distance between word being predicted and the current word (defaults to 5)
- min_count: ignores all words that have a frequency lower than this value (defaults to 5)
- workers: number of worker threads used to train the model (defaults to 3)
- sg: training algorithm; 1 for skip-gram and 0 for CBOW (defaults to 0)

In [7]:
from gensim.models import Word2Vec

word2vec_model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=3, sg=0)

Now that the model is trained we can:

1. Query for the word embeddings of a given word. 
2. Inspect the model vocabulary
3. Save the word embeddings

In [8]:
# 1. Let's see the word embedding for "apple" by accessing the "wv" attribute and passing in "apple" as the key.
print("Embedding for apple:", word2vec_model.wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words
print("\nFirst 30 vocabulary words:", list(word2vec_model.wv.vocab)[:20])

# 3. Save the word embeddings. We can save as binary format (to save space) or ASCII format
word2vec_model.wv.save_word2vec_format(SAVE_FILES_PATH+"word2vec_model", binary=True)  # binary format
word2vec_model.wv.save_word2vec_format(SAVE_FILES_PATH+"word2vec_model", binary=False)  # ASCII format

Embedding for apple: [-0.09213913 -0.02462959 -0.11255068  0.11652157 -0.18142793 -0.17555593
  0.07121698  0.086779   -0.03097944 -0.01890221 -0.04537104 -0.10696206
  0.02276987  0.08645772  0.09701958 -0.22489007  0.03993007 -0.0748188
  0.0185363  -0.257262    0.06551826  0.01579769 -0.18179104 -0.22390445
 -0.06907904 -0.08859113  0.00603421 -0.01953833 -0.0306666  -0.20717207
 -0.07466035 -0.10690664 -0.06131361 -0.0747569  -0.03541371 -0.02307771
 -0.04890924  0.09401437  0.14955166  0.03299814 -0.20348735  0.1091179
 -0.05915498  0.07897269 -0.0392515  -0.1337506   0.16920352  0.00084969
  0.09151786 -0.07067705 -0.00130636 -0.00040609 -0.09070218 -0.05848758
  0.01417456  0.12759478  0.06773403 -0.03618362  0.05180905 -0.03987553
  0.15119544  0.1374909  -0.2100861  -0.12180148 -0.01784294  0.09922534
 -0.01852375  0.2757332  -0.07551172  0.06188574 -0.0189024   0.08390908
  0.06324708 -0.02126443  0.07884526 -0.06014811 -0.1291807   0.03968196
 -0.00395843 -0.05398612  0.2568

## fastText

fastText is an unsupervised algorithm created by Facebook Research for efficiently learning word embeddings. fastText is significantly different than word2vec or GloVe in that these two algorithms treat each word as the smallest possible unit to find an embedding for. Conversely, fastText assumes that words are formed by an n-gram of characters (i.e. 2-grams of the word "language" would be {la, an, ng, gu, ua, ag, ge}). The embedding for a word is then composed of the sum of these character n-grams. This has advantages when finding word embeddings for rare words and words not present in the dictionary, as these words can still be broken down into character n-grams. Typically, for smaller datasets, fastText performs better than word2vec or GloVe. See this [tutorial](https://fasttext.cc/docs/en/unsupervised-tutorial.html) on fastText for more detail.

The gensim fastText model has many different parameters (see [here](https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText)) but the ones that are useful to know about are:  
- size: length of the word embedding/vector (defaults to 100)
- window: maximum distance between word being predicted and the current word (defaults to 5)
- min_count: ignores all words that have a frequency lower than this value (defaults to 5)
- workers: number of worker threads used to train the model (defaults to 3)
- sg: training algorithm- 1 for skip-gram and 0 for CBOW (defaults to 0)
- iter: number of epochs (defaults to 5)


In [9]:
from gensim.models.fasttext import FastText

fastText_model = FastText(size=100, window=5, min_count=5, sentences=sentences, iter=5)

We can utilize the same attributes as we saw above for word2vec due to them both originating from the gensim package

In [10]:
# 1. Let's see the word embedding for "apple" by accessing the "wv" attribute and passing in "apple" as the key.
print("Embedding for apple:", fastText_model.wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words
print("\nFirst 30 vocabulary words:", list(fastText_model.wv.vocab)[:20])

# 3. Save the word embeddings. We can save as binary format (to save space) or ASCII format
fastText_model.wv.save_word2vec_format(SAVE_FILES_PATH+"fastText_model", binary=True)  # binary format
fastText_model.wv.save_word2vec_format(SAVE_FILES_PATH+"fastText_model", binary=False)  # ASCII format

Embedding for apple: [-2.1927688e-01  2.9813698e-02  6.7616858e-02  3.6836052e-01
  2.9166859e-01 -4.3027815e-01 -4.3850473e-01 -5.5472869e-01
 -2.4860071e-01 -2.8481758e-01 -8.5550338e-02  2.0373566e-01
 -8.8941768e-02 -3.5824496e-01 -7.3820040e-02  1.9162497e-01
  1.9164029e-01  3.2222369e-01  1.7169371e-01 -1.8063694e-01
 -2.5478544e-02  3.8527763e-01 -4.4661409e-01 -1.9077049e-01
  6.3831955e-01 -3.4981030e-01  3.6546609e-01  7.3591776e-02
 -1.7809562e-01 -3.0694399e-02 -6.5486156e-04  2.8458415e-02
 -1.4853548e-01 -1.1247496e-01  2.6613681e-02 -1.5886196e-01
  1.0738261e-01  5.2269661e-01  4.1452998e-01 -2.4978566e-01
 -3.6866227e-01  4.5613028e-02 -2.5554851e-01 -2.9870963e-02
 -3.4256181e-01  4.1204464e-01  3.3703518e-01 -5.3163689e-01
  2.7413066e-02 -3.2481736e-01 -2.1018679e-01 -3.5171476e-01
  5.6522321e-02  3.2140371e-01 -3.0404109e-01  7.3594677e-01
 -4.7126335e-01  2.5894231e-01 -2.6430738e-01 -1.1617108e-01
 -2.7015641e-01 -3.2107431e-01  8.0991395e-02 -1.8977067e-01
  1

## GloVe

GloVe is an unsupervised algorithm for obtaining word embeddings created by the Stanford NLP group. Training occurs on word-word co-occurrence statistics with the objective of learning word embeddings such that the dot product of two words' embeddings is equal to the words' probability of co-occurrence. See this [tutorial](https://nlp.stanford.edu/projects/glove/) on GloVe for more detailed background on the model. 

Gensim doesn't have an implementation of the GloVe model, so we suggest getting the code directly from Stanford NLP, who developed the algorithm (https://github.com/stanfordnlp/GloVe). Run the following commands to clone the repo and then make. Clone the repo in the same location as this notebook! Otherwise, the paths below will need to be modified.  

    git clone http://github.com/stanfordnlp/glove    
    cd glove && make  

### Train GloVe vectors

Training GloVe embeddings requires some data prep and then 4 steps (also documented in the original Stanford NLP repo [here](https://github.com/stanfordnlp/GloVe/tree/master/src)).

**Step 0: Prepare Data**
   
In order to train our GloVe vectors, we first need to save our corpus as a text file with all words separated by 1+ spaces or tabs. Each document/sentence is separated by a new line character.

In [11]:
#save our corpus as tokens delimited by spaces with new line characters in between sentences
with open(BASE_DATA_PATH+'/clean/stsbenchmark/training-corpus-cleaned.txt', 'w', encoding='utf8') as file:
    for sent in sentences:
        file.write(" ".join(sent) + "\n")

**Step 1: Build Vocabulary**

Run the vocab_count executable. There are 3 optional parameters:
1. min-count: lower limit on how many times a word must appear in dataset. Otherwise the word is discarded from vocabulary.
2. max-vocab: upper bound on the number of vocabulary words to keep
3. verbose: 0, 1, or 2 (default)

Then provide the path to the text file we created in Step 0 followed by a file path that we'll save the vocabulary to 

In [12]:
!"glove/build/vocab_count" -min-count 5 -verbose 2 <"../../../data/clean/stsbenchmark/training-corpus-cleaned.txt"> "../../../data/trained_word_embeddings/vocab.txt"

BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.Processed 129989 tokens.
Counted 11986 unique words.
Truncating vocabulary at min count 5.
Using vocabulary of size 3166.



**Step 2: Construct Word Co-occurrence Statistics**

Run the cooccur executable. There are many optional parameters, but we list the top ones here:
1. symmetric: 0 for only looking at left context, 1 (default) for looking at both left and right context
2. window-size: number of context words to use (default 15)
3. verbose: 0, 1, or 2 (default)
4. vocab-file: path/name of the vocabulary file created in Step 1
5. memory: soft limit for memory consumption, default 4
6. max-product: limit the size of dense co-occurrence array by specifying the max product (integer) of the frequency counts of the two co-occurring words

Then provide the path to the text file we created in Step 0 followed by a file path that we'll save the co-occurrences to

In [13]:
!"glove/build/cooccur" -memory 4 -vocab-file "../../../data/trained_word_embeddings/vocab.txt" -verbose 2 -window-size 15 <"../../../data/clean/stsbenchmark/training-corpus-cleaned.txt"> "../../../data/trained_word_embeddings/cooccurrence.bin"

COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 13752509
overflow length: 38028356
Reading vocab from file "../../../data/trained_word_embeddings/vocab.txt"...loaded 3166 words.
Building lookup table...table contains 10023557 elements.
Processing token: 0100000Processed 129989 tokens.
Writing cooccurrences to disk.......2 files in total.
Merging cooccurrence files: processed 0 lines.0 lines.100000 lines.200000 lines.300000 lines.Merging cooccurrence files: processed 340325 lines.



**Step 3: Shuffle the Co-occurrences**

Run the shuffle executable. The parameters are as follows:
1. verbose: 0, 1, or 2 (default)
2. memory: soft limit for memory consumption, default 4
3. array-size: limit to the length of the buffer which stores chunks of data to shuffle before writing to disk

Then provide the path to the co-occurrence file we created in Step 2 followed by a file path that we'll save the shuffled co-occurrences to

In [14]:
!"glove/build/shuffle" -memory 4 -verbose 2 <"../../../data/trained_word_embeddings/cooccurrence.bin"> "../../../data/trained_word_embeddings/cooccurrence.shuf.bin"

SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 340325 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.340325 lines.Merging temp files: processed 340325 lines.



**Step 4: Train GloVe model**

Run the glove executable. There are many parameter options, but the top ones are listed below:
1. verbose: 0, 1, or 2 (default)
2. vector-size: dimension of word embeddings (50 is default)
3. threads: number threads, default 8
4. iter: number of iterations, default 25
5. eta: learning rate, default 0.05
6. binary: whether to save binary format (0: text = default, 1: binary, 2: both)
7. x-max: cutoff for weighting function, default is 100
8. vocab-file: file containing vocabulary as produced in Step 1
9. save-file: filename to save vectors to 
10. input-file: filename with co-occurrences as returned from Step 3

In [15]:
!"glove/build/glove" -save-file "../../../data/trained_word_embeddings/GloVe_vectors" -threads 8 -input-file \
"../../../data/trained_word_embeddings/cooccurrence.shuf.bin" -x-max 10 -iter 15 -vector-size 50 -binary 2 \
-vocab-file "../../../data/trained_word_embeddings/vocab.txt" -verbose 2

TRAINING MODEL
Read 340325 lines.
Initializing parameters...done.
vector size: 50
vocab size: 3166
x_max: 10.000000
alpha: 0.750000
04/30/19 - 10:33.02AM, iter: 001, cost: 0.098433
04/30/19 - 10:33.02AM, iter: 002, cost: 0.084675
04/30/19 - 10:33.02AM, iter: 003, cost: 0.074585
04/30/19 - 10:33.02AM, iter: 004, cost: 0.071048
04/30/19 - 10:33.02AM, iter: 005, cost: 0.067768
04/30/19 - 10:33.02AM, iter: 006, cost: 0.064212
04/30/19 - 10:33.02AM, iter: 007, cost: 0.060040
04/30/19 - 10:33.02AM, iter: 008, cost: 0.055310
04/30/19 - 10:33.02AM, iter: 009, cost: 0.050727
04/30/19 - 10:33.02AM, iter: 010, cost: 0.046803
04/30/19 - 10:33.02AM, iter: 011, cost: 0.043456
04/30/19 - 10:33.02AM, iter: 012, cost: 0.040570
04/30/19 - 10:33.02AM, iter: 013, cost: 0.038074
04/30/19 - 10:33.02AM, iter: 014, cost: 0.035818
04/30/19 - 10:33.02AM, iter: 015, cost: 0.033807


### Inspect Word Vectors

Like we did above for the word2vec and fastText models, let's now inspect our word embeddings

In [16]:
#load in the saved word vectors
glove_wv = {}
with open("../../../data/trained_word_embeddings/GloVe_vectors.txt", encoding='utf-8') as f:
    for line in f:
        split_line = line.split(" ")
        glove_wv[split_line[0]] = [float(i) for i in split_line[1:]]

In [17]:
# 1. Let's see the word embedding for "apple" by passing in "apple" as the key.
print("Embedding for apple:", glove_wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words
print("\nFirst 30 vocabulary words:", list(glove_wv.keys())[:20])

Embedding for apple: [0.007199, -0.055337, -0.048813, 0.463647, 0.233898, -0.020051, 0.18876, -0.19439, 0.014477, 0.122465, -0.145506, -0.056616, -0.076315, 0.051205, -0.197457, 0.197818, 0.191692, 0.259758, -0.088431, -0.101713, -0.024687, -0.083431, -0.056415, 0.08024, 0.150831, 0.030778, -0.176252, 0.291561, -0.298596, 0.111546, 0.385694, 0.184508, -0.133928, 0.007924, 0.088849, 0.016869, -0.195535, 0.002015, -0.053591, 0.043867, -0.195157, 0.270429, -0.003891, -0.033436, 0.077898, -0.083324, -0.135095, -0.419319, -0.140611, 0.000322]

First 30 vocabulary words: ['.', 'a', 'the', 'in', ',', 'is', 'to', 'of', 'and', 'on', 'man', '-', "'s", 'with', 'for', 'at', 'woman', 'are', 'that', 'two']


# Concluding Remarks

In this notebook we have shown how to train word2vec, GloVe, and fastText word embeddings on the STS Benchmark dataset. FastText is typically regarded as the best baseline for word embeddings (see [blog](https://medium.com/huggingface/universal-word-sentence-embeddings-ce48ddc8fc3a)) and is a good place to start when generating word embeddings. Now that we generated word embeddings on our dataset, we could also repeat the baseline-deep-dive notebook using these embeddings (versus the pre-trained ones from the internet). 